# Vector Indexes

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_HTTP_URL = os.getenv("WEAVIATE_HTTP_URL")
WEAVIATE_GRPC_URL = os.getenv("WEAVIATE_GRPC_URL")
AZURE_BASE_URL = os.getenv("AZURE_BASE_URL")
AZURE_RESOURCE_NAME = os.getenv("AZURE_RESOURCE_NAME")

print(WEAVIATE_HTTP_URL)
print(WEAVIATE_GRPC_URL)
print(AZURE_BASE_URL)
print(AZURE_RESOURCE_NAME)

## Connect to Weaviate

In [ ]:
import weaviate
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_HTTP_URL,
    http_port=80,
    http_secure=False,
    grpc_host=WEAVIATE_GRPC_URL,
    grpc_port=50051,
    grpc_secure=False,
)

client.is_ready()

## Collection with HNSW index (default)

[HNSW params](https://weaviate.io/developers/weaviate/config-refs/schema/vector-index#hnsw-index-parameters)

In [ ]:
from weaviate.classes.config import Configure, VectorDistances

client.collections.delete("IndexExample")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="IndexExample",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_azure_openai(
            name="main_vector",

            model="text-embedding-3-small",
            resource_name=AZURE_RESOURCE_NAME,
            base_url=AZURE_BASE_URL,

            # HSNW example 
            vector_index_config=Configure.VectorIndex.hnsw()
        )
    ],
)

## Collection with Flat index

[Flat params](https://weaviate.io/developers/weaviate/config-refs/schema/vector-index#flat-indexes)

In [ ]:
from weaviate.classes.config import Configure, VectorDistances

client.collections.delete("IndexExample")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="IndexExample",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_azure_openai(
            name="main_vector",

            model="text-embedding-3-small",
            resource_name=AZURE_RESOURCE_NAME,
            base_url=AZURE_BASE_URL,

            # Flat example
            vector_index_config=Configure.VectorIndex.flat(
                # distance_metric=VectorDistances.COSINE, # optional
                vector_cache_max_objects=100_000,
            ),
        ),
    ],
)

## Collection with Dynamic index

[Dynamic params](https://weaviate.io/developers/weaviate/config-refs/schema/vector-index#dynamic-index-parameters)

In [ ]:
from weaviate.classes.config import Configure

client.collections.delete("IndexExample")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="IndexExample",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_azure_openai(
            name="main_vector",

            model="text-embedding-3-small",
            resource_name=AZURE_RESOURCE_NAME,
            base_url=AZURE_BASE_URL,

            # Dynamic example
            vector_index_config=Configure.VectorIndex.dynamic(
                threshold=10_000, # when to switch to HNSW

                flat=Configure.VectorIndex.flat(
                    vector_cache_max_objects=100_000,
                    # note: can also include a quantizer
                    quantizer=Configure.VectorIndex.Quantizer.bq()
                ),

                hnsw=Configure.VectorIndex.hnsw(
                    max_connections=32, # optional
                    # note: the quantizer can be different between flat and hnsw
                    quantizer=Configure.VectorIndex.Quantizer.pq()
                ),
            ),
        )
    ],
)

## The rest is the same

In [ ]:
# from data_loader import import_wiki_data
# import_wiki_data(client, "IndexExample", 25000)

In [ ]:
# index_example = client.collections.get("IndexExample")
# index_example.aggregate.over_all()

## Clean up

In [ ]:
client.collections.delete("IndexExample")

## Close the client

In [ ]:
client.close()